# Example Query for Survival Probability of 1-hop Queries

Queries our system in the form of:<br>
$P(survival\_time > X | Drug \wedge Disease)$<br>
Returned is a knowledge graph containing probability of survival time and genes sensitive to that drug in the context of the disease

In [1]:
import requests
import json
import csv

# /predicate functionality example
By running /predicates you can extract a json object with the following predicates:<br>
1.) gene_to_disease_association<br>
2.) chemical_to_disease_or_phenotypic_feature_association<br>
3.) disease_to_phenotypic_association<br>

The above predicates match the following biolink entities:<br>
1.) gene<br>
2.) drug<br>
3.) disease<br>
4.) phenotypicfeature

In [2]:
r = requests.get('http://chp.thayer.dartmouth.edu/predicates/')
json_formatted_str = json.dumps(json.loads(r.content), indent=2)
print(json_formatted_str)

{
  "gene": {
    "disease": [
      "gene_to_disease_association"
    ]
  },
  "chemical_substance": {
    "disease": [
      "chemical_to_disease_or_phenotypic_feature_association"
    ]
  },
  "disease": {
    "phenotypic_feature": [
      "disease_to_phenotypic_feature_association"
    ]
  }
}


# Build Query
constructs a json query object and can take in a survival time, a disease and a single drug

In [3]:
# Function: buildQuery
#
# Input:
# -----------
# a single drug
#
# Output:
# -----------
# A query graph that asks this probablistic question: 
# P(survival_time > X | Drug = d1 and Disease = Breast Cancer)

def buildQuery(st, disease, drug):
    
    # empty response
    reasoner_std = { "query_graph": dict(),
                     "knowledge_graph": dict(),
                     "results": list()
                   }
    # empty query graph
    reasoner_std["query_graph"] = { "edges": dict(),
                                    "nodes": dict()
                                  }
    # empty knowledge graph
    reasoner_std["knowledge_graph"] = { "edges": dict(),
                                        "nodes": dict()
                                      }
    # empty response graph
    reasoner_std["results"] = [{ "node_bindings": dict(),
                                 "edge_bindings": dict()
                              }]
    
    node_count = 0
    edge_count = 0
    
    # wildcard gene slot
    reasoner_std['query_graph']['nodes']['n{}'.format(node_count)] = { 'type':'gene',
                                                                         }
    node_count += 1
    
    # drug
    reasoner_std['query_graph']['nodes']['n{}'.format(node_count)] = { 'type':'chemical_substance',
                                                                           'curie':'{}'.format(drug[1])
                                                                         }
    node_count += 1
    
    # add in disease node
    reasoner_std['query_graph']['nodes']['n{}'.format(node_count)] = { 'type':'disease',
                                                                       'curie':'{}'.format(disease[1])
                                                                     }
    node_count += 1
    
    # link gene evidence to disease
    reasoner_std['query_graph']['edges']['e{}'.format(edge_count)] = { 'type':'gene_to_disease_association',
                                                                       'source_id': 'n{}'.format(node_count -3),
                                                                       'target_id': 'n{}'.format(node_count -1)   # should be disease node
                                                                      }
    edge_count += 1
    
    # link drug evidence to disease
    reasoner_std['query_graph']['edges']['e{}'.format(edge_count)] = { 'type':'chemical_to_disease_or_phenotypic_feature_association',
                                                                       'source_id': 'n{}'.format(node_count -2),
                                                                       'target_id': 'n{}'.format(node_count -1)  # should be disease node
                                                                     }
    edge_count += 1
            
    # add target survival node
    phenotype = ('Survival_Time', 'EFO:0000714')
    reasoner_std['query_graph']['nodes']['n{}'.format(node_count)] = { 'type': 'phenotypic_feature',
                                                                       'curie': '{}'.format(phenotype[1]),
                                                                     }
    node_count += 1
    
    # link disease to target
    reasoner_std['query_graph']['edges']['e{}'.format(edge_count)] = { 'type':'disease_to_phenotypic_feature_association',
                                                                       'source_id': 'n{}'.format(node_count-2),
                                                                       'target_id': 'n{}'.format(node_count-1),
                                                                       'properties': { 'qualifier':'>=',
                                                                                       'value': st
                                                                                     }
                                                                     }
    return reasoner_std

# Read Genes and Drugs
Functionality to read in our set of available genes and drugs with respective ensemble and chembl curie IDs.

In [4]:
def readGenes():
    with open('gene_curie_map.csv', 'r') as gene_file:
        reader = csv.reader(gene_file)
        next(reader)
        rows = [(row[0],row[1]) for row in reader]
    return rows

In [5]:
def readDrugs():
    with open('drug_curie_map.csv', 'r') as drug_file:
        reader = csv.reader(drug_file)
        next(reader)
        rows = [(row[0],row[1]) for row in reader]
    return rows

# Constructing the Query and pinging CHP
You can use the commented out functionality to check which drugs are available. Disease and dryg are passed in as a tuple shown below. Currently only breast cancer can be used as the disease.

In [6]:
# list of drugs (and curies) we can query over
#drug_list = readDrugs()

survival_time = 1000
disease = ('Breast_Cancer', 'MONDO:0007254')
drug = ('CYCLOPHOSPHAMIDE', 'CHEMBL:CHEMBL88')

query = buildQuery(survival_time, disease, drug)
payload = {'message': query}

r = requests.post('http://chp.thayer.dartmouth.edu/query/', json=payload)
chp_res = json.loads(r.content)

# Extract sensitive genes
The very first result will be for the predicted survivability. Every result thereafter contains a gene with its respective sensitivity. Sensitivty values range between -1 and 1. Genes closer to -1 can be thought of as having contributed more to the false assignment of $P(survival\_time > X | Drug \wedge Disease)$. Similarly genes closer to 1 can be thought of as having contributed more to the true assignment. Gene sensitivities are order by their absolute value.

## 1. Extract predicted survivability

In [7]:
KG = chp_res['message']['knowledge_graph']
QG = chp_res['message']['query_graph']
results = chp_res['message']['results']

# holds probability of survival
survival_result = results[0]

for qge_id in survival_result['edge_bindings'].keys():
    if QG['edges'][qge_id]['type'] == 'disease_to_phenotypic_feature_association':
        kge_id = survival_result['edge_bindings'][qge_id]['kg_id']
        survival = KG['edges'][kge_id]['has_confidence_level']
        
print("P(survival_time > {} | drug & disease):".format(survival_time),survival)

P(survival_time > 1000 | drug & disease): 0.6221198156682027


## 2. Extract sensitive gene rankings

In [8]:
KG = chp_res['message']['knowledge_graph']
QG = chp_res['message']['query_graph']
results = chp_res['message']['results']

# holds gene sensitivites
sensitivity_results = results[1:]

genes = []
for sr in sensitivity_results:
    for qge_id in sr['edge_bindings'].keys():
        if QG['edges'][qge_id]['type'] == 'gene_to_disease_association':
            kge_id = sr['edge_bindings'][qge_id]['kg_id']
            sensitivity = KG['edges'][kge_id]
            gene_curie = sensitivity['source_id']
            gene_weight = sensitivity['weight']    
    for qgn_id in sr['node_bindings'].keys():
        if QG['nodes'][qgn_id]['type'] == 'gene':
            kgn_id = sr['node_bindings'][qgn_id]['kg_id']
            gene_name = KG['nodes'][kgn_id]['name']
    genes.append((gene_name, gene_curie, gene_weight))
    
for gene in genes:
    print(gene)

('PIK3CA', 'ENSEMBL:ENSG00000121879', 0.05081300813008113)
('RYR2', 'ENSEMBL:ENSG00000198626', 0.03355916892502249)
('ROCK1', 'ENSEMBL:ENSG00000067900', -0.03288166214995473)
('RBMXL3', 'ENSEMBL:ENSG00000175718', -0.030487804878048686)
('CBFB', 'ENSEMBL:ENSG00000067955', -0.026784101174344994)
('GRID2', 'ENSEMBL:ENSG00000152208', 0.025925925925925845)
('ANKAR', 'ENSEMBL:ENSG00000151687', 0.025925925925925845)
('CCDC185', 'ENSEMBL:ENSG00000178395', -0.02439024390243895)
('WHSC1L1', 'ENSEMBL:ENSG00000147548', -0.02439024390243895)
('KLHL28', 'ENSEMBL:ENSG00000179454', -0.02439024390243895)
